In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
train_data = pd.read_csv('Dataset/train.csv')
test_data = pd.read_csv('Dataset/test.csv')
ss = pd.read_csv('Dataset/sample_submission.csv')

In [5]:
import numpy as np

In [6]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [7]:
!pip install catboost


    [30/36] Cythonizing pandas\_libs/tslibs/resolution.pyx
    [31/36] Cythonizing pandas\_libs/tslibs/strptime.pyx
    [32/36] Cythonizing pandas\_libs/tslibs/timedeltas.pyx
    [33/36] Cythonizing pandas\_libs/tslibs/timestamps.pyx
    [34/36] Cythonizing pandas\_libs/tslibs/timezones.pyx
    [35/36] Cythonizing pandas\_libs/writers.pyx
    [36/36] Cythonizing pandas\io/sas/sas.pyx
    running install
    running build
    running build_py
    creating build
    creating build\lib.win-amd64-3.8
    creating build\lib.win-amd64-3.8\pandas
    copying pandas\conftest.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\testing.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\_version.py -> build\lib.win-amd64-3.8\pandas
    copying pandas\__init__.py -> build\lib.win-amd64-3.8\pandas
    creating build\lib.win-amd64-3.8\pandas\api
    copying pandas\api\__init__.py -> build\lib.win-amd64-3.8\pandas\api
    creating build\lib.win-amd64-3.8\pandas\arrays
    copying pandas\arr

In [8]:
!pip install category_encoders

In [9]:
from sklearn.metrics import accuracy_score, f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from imblearn.over_sampling import SMOTE


In [10]:
from sklearn.model_selection import StratifiedKFold

In [11]:
#Function for running cross validation
def run_gradient_boosting(clf, fit_params, train, test, features):
  N_SPLITS = 10
  oofs = np.zeros(len(train_proc))
  preds = np.zeros((len(test_proc)))

  folds = StratifiedKFold(n_splits = N_SPLITS)

  for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train[TARGET_COL])):
    print(f'\n------------- Fold {fold_ + 1} -------------')

    ### Training Set
    X_trn, y_trn = train[features].iloc[trn_idx], target.iloc[trn_idx]

    ### Validation Set
    X_val, y_val = train[features].iloc[val_idx], target.iloc[val_idx]

    ### Test Set
    X_test = test[features]
    scaler = StandardScaler()
    _ = scaler.fit(X_trn)

    X_trn = scaler.transform(X_trn)
    X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    #print(X_trn)
    #exit(0)
    
    _ = clf.fit(X_trn, y_trn, eval_set = [(X_val, y_val)], **fit_params)

    ### Instead of directly predicting the classes we will obtain the probability of positive class.
    preds_val = clf.predict_proba(X_val)[:, 1]
    preds_test = clf.predict_proba(X_test)[:, 1]

    roc_score = roc_auc_score(y_val,preds_val)
    print("ROC for validation set is {}".format(roc_score))

    oofs[val_idx] = preds_val
    preds += preds_test / N_SPLITS


  oofs_score = roc_auc_score(target, oofs.round())
  print('ROC score for oofs is {}'.format(oofs_score))
  

  return oofs, preds

In [12]:
ID_COL,TARGET_COL = 'ID','Is_Lead'

In [13]:
def join_df(train, test):

  df = pd.concat([train, test], axis=0).reset_index(drop = True)
  features = [c for c in df.columns if c not in [ID_COL, TARGET_COL]]

  return df, features

def split_df_and_get_features(df, train_nrows):

  train, test = df[:train_nrows].reset_index(drop = True), df[train_nrows:].reset_index(drop = True)
  features = [c for c in train.columns if c not in [ID_COL, TARGET_COL]]
  
  return train, test, features

In [14]:
df, features = join_df(train_data,test_data)
df.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0.0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0.0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0.0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0.0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0.0


In [15]:
#Feature engineering on the complete dataset
df['occupation_mean_balance'] = df.groupby('Occupation')['Avg_Account_Balance'].transform('mean')
df['is_active_avg_balance'] = df.groupby('Is_Active')['Avg_Account_Balance'].transform('mean')
df['gender_avg_balance'] = df.groupby('Gender')['Avg_Account_Balance'].transform('mean')
df['gender_avg_age'] = df.groupby('Gender')['Age'].transform('mean')
df['gender_and_channelcode'] = df['Gender'].astype('str') + df['Channel_Code'].astype('str')
df['gender_and_occupation'] = df['Gender'].astype('str') + df['Occupation'].astype('str')
df['gender_and_active'] = df['Gender'].astype('str') + df['Is_Active'].astype('str')



In [16]:
from category_encoders import CountEncoder

In [17]:
#dropping Avg account balance and preparin the datset for training and validation
df.drop('Avg_Account_Balance',axis=1,inplace=True)

categorical_col = ['gender_and_active','Region_Code','Gender','Credit_Product','Is_Active','Occupation','Channel_Code','gender_and_channelcode','gender_and_occupation']
#Applying feature encoding
ce = CountEncoder()
count_encoded = ce.fit_transform(df[categorical_col])
df = df.join(count_encoded.add_suffix("_count"))
df.drop(categorical_col,axis=1,inplace=True)


ID_COL , TARGET_COL = 'ID','Is_Lead'
features = [c for c in train_data.columns if c not in [ID_COL,TARGET_COL]]

train_proc, test_proc , features = split_df_and_get_features(df,train_data.shape[0])


target = train_data[TARGET_COL]


In [18]:
#catboost model training
clf = CatBoostClassifier(n_estimators = 3000,
                       learning_rate = 0.02,
                       rsm = 0.4, ## Analogous to colsample_bytree
                       random_state=2054,
                       
                       )

fit_params = {'verbose': 200, 'early_stopping_rounds': 300}

cb_oofs, cb_preds = run_gradient_boosting(clf, fit_params, train_proc, test_proc, features)

optimized_roc = roc_auc_score(target, (cb_oofs  * 1))
print(f'Optimized ROC is {optimized_roc}')


------------- Fold 1 -------------
0:	learn: 0.6760873	test: 0.6762091	best: 0.6762091 (0)	total: 143ms	remaining: 7m 8s
200:	learn: 0.3459397	test: 0.3481982	best: 0.3481982 (200)	total: 16s	remaining: 3m 43s
400:	learn: 0.3432245	test: 0.3458744	best: 0.3458744 (400)	total: 30.7s	remaining: 3m 19s
600:	learn: 0.3420228	test: 0.3452067	best: 0.3452067 (600)	total: 45.9s	remaining: 3m 3s
800:	learn: 0.3409732	test: 0.3447860	best: 0.3447810 (792)	total: 1m 2s	remaining: 2m 50s
1000:	learn: 0.3401011	test: 0.3446186	best: 0.3446185 (999)	total: 1m 17s	remaining: 2m 34s
1200:	learn: 0.3393083	test: 0.3445615	best: 0.3445520 (1189)	total: 1m 32s	remaining: 2m 19s
1400:	learn: 0.3386240	test: 0.3445061	best: 0.3444846 (1341)	total: 1m 48s	remaining: 2m 4s
1600:	learn: 0.3379580	test: 0.3444771	best: 0.3444739 (1595)	total: 2m 3s	remaining: 1m 47s
1800:	learn: 0.3373622	test: 0.3445350	best: 0.3444739 (1595)	total: 2m 19s	remaining: 1m 32s
Stopped by overfitting detector  (300 iterations w

800:	learn: 0.3416141	test: 0.3387166	best: 0.3387166 (800)	total: 1m	remaining: 2m 47s
1000:	learn: 0.3407234	test: 0.3385825	best: 0.3385825 (1000)	total: 1m 16s	remaining: 2m 33s
1200:	learn: 0.3399427	test: 0.3385540	best: 0.3385340 (1115)	total: 1m 30s	remaining: 2m 16s
1400:	learn: 0.3392106	test: 0.3385682	best: 0.3385340 (1115)	total: 1m 45s	remaining: 2m
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3385340331
bestIteration = 1115

Shrink model to first 1116 iterations.
ROC for validation set is 0.8766764433365811

------------- Fold 10 -------------
0:	learn: 0.6761337	test: 0.6762012	best: 0.6762012 (0)	total: 63.6ms	remaining: 3m 10s
200:	learn: 0.3462503	test: 0.3462677	best: 0.3462677 (200)	total: 13.7s	remaining: 3m 11s
400:	learn: 0.3433735	test: 0.3438088	best: 0.3438088 (400)	total: 27.1s	remaining: 2m 55s
600:	learn: 0.3422772	test: 0.3432089	best: 0.3432089 (600)	total: 42s	remaining: 2m 47s
800:	learn: 0.3412696	test: 0.3427951	best: 0.342793

In [19]:
#training LightGBM model 
clf = LGBMClassifier(n_estimators = 200,
                        learning_rate = 0.05,
                        colsample_bytree = 0.5,
                        )
fit_params = {'verbose': 100, 'early_stopping_rounds': 100}

lgb_oofs, lgb_preds = run_gradient_boosting(clf, fit_params, train_proc, test_proc, features)


optimized_roc = roc_auc_score(target, (lgb_oofs * 1))
print(f'Optimized ROC is {optimized_roc}')


------------- Fold 1 -------------
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.34596
[200]	valid_0's binary_logloss: 0.344513
Did not meet early stopping. Best iteration is:
[195]	valid_0's binary_logloss: 0.344503
ROC for validation set is 0.873035109783767

------------- Fold 2 -------------
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.345418
[200]	valid_0's binary_logloss: 0.344206
Did not meet early stopping. Best iteration is:
[196]	valid_0's binary_logloss: 0.344194
ROC for validation set is 0.8725648088900428

------------- Fold 3 -------------
Training until validation scores don't improve for 100 rounds
[100]	valid_0's binary_logloss: 0.346475
[200]	valid_0's binary_logloss: 0.345006
Did not meet early stopping. Best iteration is:
[200]	valid_0's binary_logloss: 0.345006
ROC for validation set is 0.8736084046923609

------------- Fold 4 -------------
Training until validatio

In [20]:
#training XGB Classifier
clf = XGBClassifier(n_estimators = 1000,
                    max_depth = 6,
                    learning_rate = 0.05,
                    colsample_bytree = 0.5,
                    random_state=1452,
                    )

fit_params = {'verbose': 200, 'early_stopping_rounds': 200}

xgb_oofs, xgb_preds = run_gradient_boosting(clf, fit_params, train_proc, test_proc, features)


optimized_f1 = roc_auc_score(target, (xgb_oofs * 1))
print(f'Optimized F1 is {optimized_f1}')


------------- Fold 1 -------------


C:\Users\ekajgho\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:29:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.66577
[200]	validation_0-logloss:0.34477
[400]	validation_0-logloss:0.34471
[514]	validation_0-logloss:0.34498
ROC for validation set is 0.8727225948421754

------------- Fold 2 -------------


C:\Users\ekajgho\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:29:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.66546
[200]	validation_0-logloss:0.34455
[400]	validation_0-logloss:0.34452
[479]	validation_0-logloss:0.34464
ROC for validation set is 0.8723073421580341

------------- Fold 3 -------------


...

[18:33:43] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.66560
[200]	validation_0-logloss:0.34171
[400]	validation_0-logloss:0.34169
[468]	validation_0-logloss:0.34184
ROC for validation set is 0.8755250144419445

------------- Fold 9 -------------


C:\Users\ekajgho\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:34:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.66542
[200]	validation_0-logloss:0.33885
[400]	validation_0-logloss:0.33862
[573]	validation_0-logloss:0.33881
ROC for validation set is 0.8768158034217604

------------- Fold 10 -------------


C:\Users\ekajgho\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[18:35:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.66565
[200]	validation_0-logloss:0.34309
[400]	validation_0-logloss:0.34297
[459]	validation_0-logloss:0.34303
ROC for validation set is 0.8757283590711745
ROC score for oofs is 0.7538187358814215
Optimized F1 is 0.8743666729190007


In [21]:
#preparing the input from CatBoost, LightGBM , XGBoost
train_new = train_data[[ID_COL, TARGET_COL]]
test_new = test_data[[ID_COL]]

train_new['lgb'] = lgb_oofs
test_new['lgb'] = lgb_preds

train_new['cb'] = cb_oofs
test_new['cb'] = cb_preds

train_new['xgb'] = xgb_oofs
test_new['xgb'] = xgb_preds

features = [c for c in train_new.columns if c not in [ID_COL, TARGET_COL]]

<ipython-input-21-2bb1dc51abc8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_new['lgb'] = lgb_oofs
<ipython-input-21-2bb1dc51abc8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_new['lgb'] = lgb_preds
<ipython-input-21-2bb1dc51abc8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [22]:
#final ensembled model
clf = CatBoostClassifier(n_estimators = 3000,
                       learning_rate = 0.04,
                       rsm = 0.4, ## Analogous to colsample_bytree
                       random_state=2054,
                       
                       )

fit_params = {'verbose': 200, 'early_stopping_rounds': 300}

cb_oofs, cb_preds = run_gradient_boosting(clf, fit_params, train_new, test_new, features)

optimized_f1 = roc_auc_score(target, (cb_oofs  * 1))
print(f'Optimized ROC is {optimized_f1}')


------------- Fold 1 -------------
0:	learn: 0.6811419	test: 0.6811420	best: 0.6811420 (0)	total: 21.8ms	remaining: 1m 5s
200:	learn: 0.3430342	test: 0.3443613	best: 0.3443613 (200)	total: 8.2s	remaining: 1m 54s
400:	learn: 0.3425902	test: 0.3443580	best: 0.3443443 (226)	total: 16.2s	remaining: 1m 45s
600:	learn: 0.3421755	test: 0.3443052	best: 0.3442901 (527)	total: 24.9s	remaining: 1m 39s
800:	learn: 0.3417751	test: 0.3443414	best: 0.3442901 (527)	total: 33.8s	remaining: 1m 32s
Stopped by overfitting detector  (300 iterations wait)

bestTest = 0.3442900942
bestIteration = 527

Shrink model to first 528 iterations.
ROC for validation set is 0.8734233669247767

------------- Fold 2 -------------
0:	learn: 0.6811419	test: 0.6811420	best: 0.6811420 (0)	total: 33.3ms	remaining: 1m 39s
200:	learn: 0.3430059	test: 0.3440408	best: 0.3440408 (200)	total: 8.12s	remaining: 1m 53s
400:	learn: 0.3426166	test: 0.3439205	best: 0.3439205 (400)	total: 16.5s	remaining: 1m 46s
600:	learn: 0.3421717	te

In [26]:
cb_preds_rounded = (cb_preds) * 1
# download_preds(cb_preds_rounded, file_name = 'final.csv')
ss[TARGET_COL] = cb_preds_rounded

In [27]:
ss

,ID,Is_Lead
0,VBENBARO,0.046057
1,CCMEWNKY,0.837323
2,VK3KGA9M,0.052817
3,TT8RPZVC,0.025635
4,SHQZEYTZ,0.019318
...,...,...
105307,DBENJOYI,0.988357
105308,CWQ72DWS,0.591613
105309,HDESC8GU,0.079910
105310,2PW4SFCA,0.207653


In [28]:
ss.to_csv('sub.csv',index=False,header=True)